In [5]:
# Imports

import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import re

In [2]:
# Set up Splinter
executable_path = {'executable_path': 'chromedriver.exe'}

In [32]:
# Soupify the NASA Mars url
url = f"https://mars.nasa.gov/news/"
with Browser('chrome', **executable_path, headless=False) as browser:
    browser.visit(url)
    soup = bs(browser.html, 'html.parser')


# Scrape the relevant data
newsTitle = soup.find("div", class_="content_title").text
newsDescription = soup.find("div", class_="article_teaser_body").text

print(newsTitle)
print(newsDescription)

What's Mars Solar Conjunction, and Why Does It Matter?
NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.


In [3]:
# Visit the JPL site
jplUrl = f"https://www.jpl.nasa.gov"
marsImagesUrlParam = "/spaceimages/?search=&category=Mars"

with Browser('chrome', **executable_path, headless=False) as browser:
    browser.visit(jplUrl + marsImagesUrlParam)
    soup = bs(browser.html, 'html.parser')
    
imageUrl = soup.find("a", class_="button fancybox")['data-fancybox-href']
featured_image_url = jplUrl + imageUrl

print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA14417_ip.jpg


In [6]:
# Mars Weather
marsWeatherTwitterUrl = f"https://twitter.com/marswxreport?lang=en"

with Browser('chrome', **executable_path, headless=False) as browser:
    browser.visit(marsWeatherTwitterUrl)
    soup = bs(browser.html, 'html.parser')

# Scrape the weather - Gets the last weather tweet, not some other news tweets
mars_weather = soup.find('p', text = re.compile('InSight'), 
                       attrs = {'class' : 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text'}).text
print(mars_weather)

InSight sol 263 (2019-08-23) low -100.9ºC (-149.6ºF) high -27.2ºC (-17.0ºF)
winds from the SW at 4.1 m/s (9.2 mph) gusting to 18.3 m/s (40.9 mph)
pressure at 7.60 hPa


In [5]:
# Mars Facts
marsFactsUrl = f"https://space-facts.com/mars/"
marsFacts = pd.read_html(marsFactsUrl)
marsFacts = marsFacts[1]
marsFacts.columns = ["Description", "Value"]
marsFacts.set_index("Description", inplace = True)
marsFacts

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [66]:
# Mars Hemispheres

astrogeologyUrl = f"https://astrogeology.usgs.gov"
searchUrl = f"/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
mainPageUrl = astrogeologyUrl + searchUrl

imageList = []

with Browser('chrome', **executable_path, headless=False) as browser:
    browser.visit(mainPageUrl)
    soup = bs(browser.html, 'html.parser')
    
    hemispheresLinks = soup.find_all("div", class_="description")
    for link in hemispheresLinks:
        hemisphereUrl = astrogeologyUrl + link.a["href"]
        browser.visit(hemisphereUrl)
        soup = bs(browser.html, 'html.parser')
        image = astrogeologyUrl + soup.find("img", class_="wide-image")["src"]
        title = soup.find("h2", class_="title").text.replace(" Enhanced", "")
        imageList.append( { "title" : title, "img_url" : image } )

imageList

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]